<a href="https://colab.research.google.com/github/kuruvilla2087/30daysofML/blob/master/CIFAR_10_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [42]:
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation,GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,EarlyStopping,ModelCheckpoint 
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import math

In [43]:
# Hyperparameters
batch_size = 64
num_classes = 10
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [44]:
# Load CIFAR10 Data
(x_train,y_train), (x_test,y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [45]:
# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [46]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


# Normalising Data

It is required to normalize the data so that all features come to a scale between 0 and 1. This helps better understanding by the network. It is also required to convert the data type of individual pixel value to float before normalization because float is more accurate than integers.

**Caution:** If data type in not converted to float before dividing, then we may end up ceiling the data if proper typecast is not done.


In [47]:
# # normalize inputs from 0-255 to 0.0-1.0
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train = x_train / 255.0
# x_test = x_test / 255.0



# Data Augmentation

Data Augmentation is a great method to **indirectly acquire more data** from the already availabe data. This happens by horizontal or verticle flips, rotation of images, random cropping and many more image processing ways such as whitening, scaling and different channels improvization. 
Helps in generalization process


# Dense Block

This is the actual feature extracter block.

   1.  Architecture has **3 dense blocks**.
    Batch Normalization helps in **equalising the outcome values** from convolution.
    2. Activation function: **RELU**
    3. **Dropout** is done after 3x3 convolution for **regularization purpose** where 20% of neurons are randomly shut down to avoid overfitting on train data.
    4. **1x1 Convolution** is done to to reduce the number of channels before giving it to next layer so that it helps in achieving lower number of parameters and reducing model complexity.
    5. Standard **3x3 Convolution** is followed.
    6. The number of filters used in 1x1 is **four times(4x) the number of kernals** in 3x3 convolution as advised by the above mentioned paper. This helps by not losing too much of information when reduing the number of channels.



In [48]:
# Dense Block
def denseblock(input, num_filter = 16, dropout_rate = 0.25):
    global compression
    temp = input
    for _ in range(l):
      
        BatchNorm = BatchNormalization()(temp)
        reluD_LAYER = Activation('relu')(BatchNorm)
        Conv2D_1_1 = Conv2D(int(4*num_filter*compression), (1,1), use_bias=False ,kernel_initializer="he_uniform",padding='same')(reluD_LAYER)
        if dropout_rate>0:
          Conv2D_1_1 = Dropout(dropout_rate)(Conv2D_1_1)
                    
        BatchNorm = BatchNormalization()(Conv2D_1_1)
        reluD_LAYER = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,kernel_initializer="he_uniform",padding='same')(reluD_LAYER)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
def transition(input, num_filter = 16, dropout_rate = 0.25):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(4*num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg    
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    GblAvgPooling = GlobalAveragePooling2D()(relu)
    #flat = Flatten()(GblAvgPooling)
    output = Dense(num_classes, activation='softmax')(GblAvgPooling)
    
    return output    

# Model implementation

In [49]:

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

l = 16
First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

l = 16
Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

l = 16
Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [50]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_396 (Conv2D)             (None, 32, 32, 12)   324         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_396 (BatchN (None, 32, 32, 12)   48          conv2d_396[0][0]                 
__________________________________________________________________________________________________
activation_396 (Activation)     (None, 32, 32, 12)   0           batch_normalization_396[0][0]    
_______________________________________________________________________________________

In [51]:
#callbacks
from time import time
from datetime import datetime


from tensorflow.python.keras.callbacks import TensorBoard

filepath = "weights.{epoch:02d}-{val_loss:.2f}.hdf5"
history = tf.keras.callbacks.History()

# tensorboard
tensorboard = TensorBoard(log_dir="model_logs/{}".format(time()))

filepath = "weights.{epoch:02d}-{val_loss:.2f}.hdf5"
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
checkpoint_save = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint_save,learning_rate_reduction,history,tensorboard]

In [52]:
epochs = 100
batch_size = 64
steps = len(x_train)//batch_size

In [53]:
sgd = tf.keras.optimizers.SGD(lr = 0.1,momentum = 0.9,nesterov = True)
model.compile(sgd,loss="categorical_crossentropy",metrics=["accuracy"])

In [54]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    zoom_range=0.3,
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    fill_mode='nearest')

datagen.fit(x_train)

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),steps_per_epoch=steps,
                    epochs=100,callbacks=callbacks_list,
                    validation_data=(x_test,y_test))

Epoch 1/100
781/781 [==============================] - 98s 125ms/step - loss: 1.7641 - accuracy: 0.3467 - val_loss: 1.7769 - val_accuracy: 0.3758
Epoch 2/100
781/781 [==============================] - 95s 122ms/step - loss: 1.4675 - accuracy: 0.4629 - val_loss: 1.7534 - val_accuracy: 0.3736
Epoch 3/100
781/781 [==============================] - 95s 122ms/step - loss: 1.3102 - accuracy: 0.5282 - val_loss: 1.4225 - val_accuracy: 0.5202
Epoch 4/100
781/781 [==============================] - 95s 122ms/step - loss: 1.1959 - accuracy: 0.5705 - val_loss: 1.9301 - val_accuracy: 0.4961
Epoch 5/100
781/781 [==============================] - 95s 121ms/step - loss: 1.1094 - accuracy: 0.6030 - val_loss: 1.4003 - val_accuracy: 0.5737
Epoch 6/100
781/781 [==============================] - 95s 122ms/step - loss: 1.0490 - accuracy: 0.6274 - val_loss: 1.2444 - val_accuracy: 0.6234
Epoch 7/100
781/781 [==============================] - 94s 120ms/step - loss: 1.0042 - accuracy: 0.6447 - val_loss: 1.1140 -